# Mini-lab 4 - Mapping TAZ travel times

In this lab we will play with some data from the Metropolitan Transportation Commission on travel time from one Traffic Analysis Zone (TAZ) to another. We will use a mapping tool called folium to create a graphical representation of travel times throughout the area.


In [1]:
from datascience import *
import numpy as np

## The datasets
### MTC travel skims
The Metropolitan Transportation Co,mission (MTC) is the regional transportation planning organization for the Bay Area. They host a database with average travel time, cost, and distance from each traffic analysis zone (TAZ) to all other TAZs in the Bay Area. The files have data for driving alone, car pooling, walking to transit, driving to transit, walking, and biking. 

We have pre-processed the data from the morning commute to include only TAZs around San Francisco, Oakland and Berkeley. The files with inter-TAZ travel time, travel cost, and travel distance are saved with the following filepaths 'data/sf_oak_TimeSkims_AM.csv', 'data/sf_oak_CostSkims_AM.csv' and 'data/sf_oak_DistanceSkims_AM.csv'

More info on the dataset can be found here - http://analytics.mtc.ca.gov/foswiki/Main/SimpleSkims. 
The descriptions of the columns in the data set are shown below:

    orig	Origin transportation analysis zone	Shape file
    dest	Destination transportation analysis zone	Shape file
    da	    Door-to-door time for the drive alone travel mode (i.e. single occupant private automobile)	 
    s2	    Door-to-door time for the shared ride 2 travel mode (i.e. double occupant private automobile)	 
    s3	    Door-to-door time for the shared ride 3+ travel mode (i.e. three-or-more occupants traveling in a private vehicle)	 
    walk	Door-to-door time for walking	 
    bike	Door-to-door time for bicycling	 
    wTrnW	Door-to-door time for walk to transit to walk paths	 
    dTrnW	Door-to-door time for drive to transit to walk paths	 
    wTrnD	Door-to-door time for walk to transit to drive paths (returning home on a park-and-ride tour)

(The raw data with all bay area TAZs can be found at https://mtcdrive.app.box.com/2015-03-116)

### Bay area TAZ geometry data
GeoJSON is a format used for encoding a variety of geographic data structures. We have saved a GeoJSON file with the Traffic Analysis Zone (TAZ) polygons for the TAZs in the San Francisco, Oakland, and Berkeley region. We will use a mapping package called folium to map the TAZs.

### Read the data
First we read in the sf_oak_TimeSkims_AM.csv using the Table function

In [2]:
# Read the timeskims datafile
tt_data = Table.read_table('data/sf_oak_traveltimes_bymode.csv')
tt_data

OSError: File b'data/sf_oak_traveltimes_bymode.csv' does not exist

In [ ]:
# Question: what is the drive alone travel time from origin TAZ 10 (in downtown SF) to destination TAZ 
# 1019 (the TAZ at UC Berkeley) according to this dataset. Hint: use a where() statement to find the answer.

#Answer key:
tt_data.where('origin', 10).where('destination', 1019)['drive alone']

# Use the datascience map tool to map the TAZs
As mentioned above, geojson is a standard format for encoding geographic data. 'SF_Oak_TAZs.geojson' is a geojson file that contains the TAZ geometry for TAZs in the San Francisco and Oakland area. We can use the datascience Map tool to map the polygons in the geojson file.

In [ ]:
base_taz_map = Map.read_geojson('../../demos/data/SF_Oak_TAZs.geojson')
base_taz_map

# Create a Table of the map features
The geojson contains a few hundred unique polygon features, one for each TAZ. In the cell below we can use the Map objects .map_features property to get a list of the polygon features. Then we can use the Table.from_records() method to convert this to a table.

The corresponding table has 5 total columns, 3 corresponding to the properties in the geojson feature:
* County_FIP = An id for the county a TAZ is contained in
* TAZ1454 = A numeric TAZ id.'TAZ1454' refers to the fact that in total there are 1454 TAZs in the bay area, and they are assigned ids from 1 to 1454. Only about 300 are shown in the map above.
* UrbanTAZac = TODO fill this in.

The remaining columns includs a feature column containing info on the polygon itself (it is displayed as a map in the table) and an id column, where the id is a newly assigned Table id, with values from 0 to the length of the table - 1.

In [ ]:
taz_table = Table.from_records(base_taz_map.features)

# Note - DO NOT use the Table.show() method here - it will likely make your notebook crash!
taz_table

# Using maps to convey information
## Color TAZs based on TAZ id
First, we might want to know the general location of the various TAZ ids. We can use the Map color() method to color the map based on the value in a table. To do this we:

1. Create a 2-column table, column 1 should contain the feature ids and column 2 should contain the information used to color the table.
2. Use the map.color(values, ids=(), key_on='feature.id', palette='YlOrBr') method to color the map. The inputs to this method are:
 * values = the 2-column table feature descibed in #1, 
 * key_on = to the name of the map property to be matched to the ids in the table, in this case 'feature.properties.TAZ1454'. 
 * Optional 
    * color palette: 'BuGn', 'BuPu', 'GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu', 'YlOrBr', and 'YlOrRd'
    * fill_opacity: 1 for totally opaque, 0 for totally transparent
    * threshold_scale (where the color breaks are).

In [ ]:
# 1. Create a 2-column table with columns "ids" and "values". Since we want to color based on 
# the value of the id, in this case both the value and id column are set to the TAZ1454 values. 

tocolor_table = Table().with_columns(['ids', taz_table.column('TAZ1454'), 
                                      'values', taz_table.column('TAZ1454')])
tocolor_table

In [ ]:
base_taz_map.color(values=tocolor_table,key_on='feature.properties.TAZ1454', 
              threshold_scale=[100,200,400,1000,1200],palette='YlOrRd',fill_opacity=1.)

In [ ]:
# Question: From the map above, what city would you guess the TAZ with TAZ1454=150 
# is located in?



# Conveying more interesting information

## Color the map based on the 'bike' travel time from one origin TAZ to all other TAZs.

In [ ]:
origin_taz_id = 10
mode = 'bike'

threshold_scale = [0,5,10,15,20,25] #sets where the color transitions are

#Answer key:
tt_from_orig = tt_data.where(tt_data['origin']==origin_taz_id)
tt_from_orig = tt_from_orig.select(['destination',mode])

taz_map = base_taz_map.color(tt_from_orig, key_on='feature.properties.TAZ1454', palette='YlOrRd',
                        threshold_scale=threshold_scale, fill_opacity=.8)
taz_map

In [ ]:
tt_from_orig

# Use the overlay method to color the origin TAZ blue

In [ ]:
origin_taz_id = 10

def overlay(feature, ds_map, color='Blue', opacity=.6):
    '''
    feature can be a table of map features, a list of map features,
    a Map, a Region, or a circle marker map tables. The feautres will
    be drawn on the existing ds_map. 
    '''   
    if type(feature) == Table:
        # if table of features e.g. Table.from_records(taz_map.features)
        if 'feature' in feature:
            feature = feature['feature']
            
        # if marker table e.g. table with columns: latitudes,longitudes,popup,color,radius
        elif 'radius' in feature:
            feature = Circle.map_table(feature)
        else:
            #can't overlay marker maps, so will force to be circle markers
            print ('ERROR - can only overlay cirle marker maps or regions.')

    if type(feature) in [list, np.ndarray]:
        for f in feature:
            f._attrs['fill_color'] = color
            f._attrs['fill_opacity'] = opacity
            f.draw_on(ds_map._folium_map)
    
    elif type(feature) == Map:
        for i in range(len(feature._features)):
            f = feature._features[i]
            f._attrs['fill_color'] = color
            f._attrs['fill_opacity'] = opacity
            f.draw_on(ds_map._folium_map)
    elif type(feature) == Region:
        feature._attrs['fill_color'] = color
        feature._attrs['fill_opacity'] = opacity
        feature.draw_on(ds_map._folium_map)
    return ds_map


# find the taz with TAZ1454 id = origin_taz_od by using the .where() method on the taz_table
features = taz_table.where(taz_table['TAZ1454']==origin_taz_id)['feature']

#use the overlay method to color the feature blue
overlay(features, taz_map, 'Blue', .9)

# Color unreachale TAZs Grey
For the datasets provided, MTC follows a convention of setting travel time = '-999' in the data table if they have determined that a mode is not feasible to get from an origin TAZ to a destination TAZ. For example, a bike trip from San Francisco to Oakland is not possible because bikes are not allowed accross the Bay Bridge. Similarly some walk distances are determined to be too far to make a trip on foot.
Color all TAZs with a travel time of -999 grey. Hint - use a procedure similar to the procedure used to color the origin blue.

In [ ]:
#color unreachable grey:

# Tazs that are unreachable by the selected mode have a travel time of -999
# Your task: use the .where() method on the tt_from_orig table to create a list of unreachable TAZs 


# answer key:
unreachable = tt_from_orig.where(1,-999)['destination']

# Now we need to select only the features from the taz_table that have TAZ1454 in the
# unreachable list. Use the .where function to select these features. You may want to 
# use the np.in1d() method that returns True if an element is in a list. Once we have a 
# list of features, use the overlay method to add them to our taz_map

# np.in1d example:
# np.in1d([3,5,2], [1,2,3])


features = taz_table.where(np.in1d(taz_table['TAZ1454'], unreachable))['feature']
overlay(features, taz_map, 'Grey', .8)

# Put it all in one method

Create a method that produces a travel time map. You have all of the components above, 

In [ ]:
def create_tt_map(data, taz_map, origin_taz_id=1, mode='da', color_palette='YlOrRd', 
                  threshold_scale=[5,10,15,20,25]):

    #1. create the map
    tt_from_orig = data.where(data['origin']==origin_taz_id)
    tt_from_orig = tt_from_orig.select(['destination',mode])

    taz_map = taz_map.color(tt_from_orig, key_on='feature.properties.TAZ1454', palette=color_palette,
                            threshold_scale=threshold_scale, fill_opacity=0.8)
    
    #2. create the taz_table from the map
    taz_table = Table.from_records(taz_map.features)
    
    
    #3. color the unreachable tazs grey:
    # YOUR TASK - fill this in
    unreachable = tt_from_orig.where(1,-999)['dest']
    features = taz_table.where(np.in1d(taz_table['TAZ1454'], unreachable))['feature']
    overlay(features, taz_map, 'Grey', .9)
    
    #4. color the origin blue
    # YOUR TASK  - fill this in
    features = taz_table.where(taz_table['TAZ1454']==origin_taz_id)['feature']
    overlay(features, taz_map, 'Blue', .9)
    
    return taz_map

In [ ]:
overlay(taz_table.where(taz_table['TAZ1454']==origin_taz_id), taz_map)
# taz_table.where(taz_table['TAZ1454']==origin_taz_id)

# Test your method
By running the following code. 

In [ ]:
create_tt_map(tt_data, taz_map, origin_taz_id = 917, mode='bike')

# Explore travel times for other travel modes:
Use the method you created to plot the walk transit walk ('wTrnW') travel times from TAZ 5. Try a differeny color palette and modify the threshold_scale so the color transitions occur at appropriate levels

In [ ]:
create_tt_map(tt_data, taz_map, origin_taz_id = 5, mode='wTrnW', color_palette='PuBu', 
              threshold_scale=[15,30,45,60,75])

# The relationship between spatial proximity and travel time
For each of the following modes: drive, walk to transit, drive to transit, bike, and walk, do you think a straightline distance from one TAZ to another is a good indicator of travel time? Explain why or why not.



In [ ]:
# Your answer here:



# Extra material - adding Circle marker labels to the map

In [ ]:
from shapely import geometry
import json

#create marker table with location = centroid of a TAZ and color and radius specified
features = taz_table['feature']
lats, lons, labels = [], [], []
for i in range(len(taz_table.column('id'))):
    ct = geometry.shape(features[i].geojson(0)['geometry']).centroid
    lats.append(ct.xy[1][0])
    lons.append(ct.xy[0][0])
    labels.append('TAZ '+str(taz_table['TAZ1454'][i]))

markers=Table().with_columns(['latitudes',lats, 'longitudes', lons, 'popup', labels])
markers['color']='blue'
markers['radius']=200
markers

In [ ]:

overlay(markers,taz_map, opacity=0.7)


In [ ]:
# Answer here

# Optional: Modify your code to map travel cost rather than travel time. 
Note that travel cost from one TAZ to another is saved in the file SF_OAK_CostSkims.csv.
